In [1]:
!pip install nltk
!pip install sentence-transformers
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

from sklearn import linear_model
from  sentence_transformers import SentenceTransformer

from sklearn import svm
import torch 

from scipy.stats import spearmanr

from sentence_transformers import InputExample, losses, models
from sentence_transformers import util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from torch import nn
from datetime import datetime
import math

In [3]:
df_train=pd.read_csv("H1_train.csv", sep=",", encoding="utf8")

In [4]:
df_train.head()

,tweet,label,id
0,@USER She should ask a few native Americans wh...,OFF,0
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,1
2,Amazon is investigating Chinese employees who ...,NOT,2
3,"@USER Someone should'veTaken"" this piece of sh...",OFF,3
4,@USER @USER Obama wanted liberals &amp; illega...,NOT,4


In [19]:
NON_CONEXTUAL_MODEL_TYPE = "glove.6B.50d.txt" 

In [6]:
BATCH_SIZE = 8
OUT_DIM_DENSE = 256
NUM_EPOCHS = 3 ## THIS IS FIXED DO NOT CHANGE ; changed from 2 --> 3 since MAX NUM EPOCHS = 3

MAX_SEQ_LENGTH = 256

In [7]:
GLOVE_EMBEDDINGS_DIM = 50

In [8]:
df_train["label"]=df_train["label"].replace(to_replace="OFF",value=0)
df_train["label"]=df_train["label"].replace(to_replace="NOT",value=1)

In [9]:
def preprocess(raw_text):
    list_of_words = re.sub(r"^A-Za-z", " ", raw_text).lower().split()
    return list_of_words
    
def get_sentence_embedding(sentence, embeddings_dict):
    # While creating sentence vector, only consider words that have a key in the glove dict
    
    relevant_embeddings = [embeddings_dict[word] for word in preprocess(sentence) if word in embeddings_dict.keys()]
    if len(relevant_embeddings) > 0:
        sentence_embedding = np.mean(relevant_embeddings, axis=0)
    else:
        sentence_embedding = np.zeros(GLOVE_EMBEDDINGS_DIM)
    return sentence_embedding

In [10]:
def get_model1():
    """
    return glove embeddings dict
    """
    # https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db
    embeddings_dict = dict()
    with open(NON_CONEXTUAL_MODEL_TYPE, "r", encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    
    print(len(embeddings_dict))
    return embeddings_dict

In [11]:
def get_feature_model1(data_frame):
    """
    Input a data frame and return the embedding vectors for the each sentence column using non_cont_model1,
    Return 2 matrices each of shape (#_samples, #size_of_word_emb).
    """

    # https://www.kaggle.com/code/adepvenugopal/nlp-text-similarity-using-glove-embedding?scriptVersionId=105939541&cellId=2
    S1 = np.array([get_sentence_embedding(sentence, non_cont_model1) for sentence in data_frame["tweet"]])
#     S2 = np.array([get_sentence_embedding(sentence, non_cont_model1) for sentence in data_frame["S2"]])
    
    return S1

In [13]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_val, y_train, y_val = train_test_split(df_train["tweet"], df_train["label"], test_size=0.33, random_state=42)

In [27]:
X_train.shape

(8870,)

In [28]:
y_train

10126    1
11593    1
11078    1
1963     1
3125     1
        ..
11964    1
5191     1
5390     1
860      1
7270     1
Name: label, Length: 8870, dtype: int64

In [29]:
df_test=pd.DataFrame({"tweet": X_val, "label": y_val})
df_train=pd.DataFrame({"tweet": X_train, "label": y_train})

In [30]:
non_cont_model1 = get_model1()

feature_1_train = get_feature_model1(df_train)
print(feature_1_train.shape)

# feature_1_dev, feature_2_dev = get_feature_model1(df_dev)
# print(feature_1_dev.shape, feature_2_dev.shape)

feature_1_test = get_feature_model1(df_test)
print(feature_1_test.shape)

X_train, Y_train = np.array(feature_1_train), np.array(df_train["label"])
# X_dev, Y_dev = np.hstack((feature_1_dev, feature_2_dev)), np.array(df_dev["SCORE"]).astype(np.float32
X_test, Y_test = feature_1_test, np.array(df_test["label"])

print(X_train.shape, Y_train.shape)
# print(X_dev.shape, Y_dev.shape)
print(X_test.shape, Y_test.shape)

400000
(8870, 50)
(4370, 50)
(8870, 50) (8870,)
(4370, 50) (4370,)


In [31]:
! pip install gensim

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [32]:
import pandas as pd
import numpy as np
import math
import string
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import PassiveAggressiveClassifier

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import gensim
from gensim.models import Word2Vec

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\udit1\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [33]:
logictic_model = LogisticRegression().fit(X_train,Y_train)
y_preds = logictic_model.predict(X_test)
report = classification_report( Y_test, y_preds )
print(report)
acc=accuracy_score(Y_test,y_preds)
print("Logistic Regression, Accuracy Score:" , acc)
# dict_models_tfidf["logictic_model"]=logictic_model
# dict_accuracy_report_tfidf["Logistic Regression"]=report

              precision    recall  f1-score   support

           0       0.61      0.17      0.26      1467
           1       0.69      0.95      0.80      2903

    accuracy                           0.68      4370
   macro avg       0.65      0.56      0.53      4370
weighted avg       0.66      0.68      0.62      4370

Logistic Regression, Accuracy Score: 0.6844393592677346


C:\Users\udit1\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
model_pa = PassiveAggressiveClassifier(C = 0.6, random_state = 5)
model_pa.fit(X_train, Y_train)
test_pred = model_pa.predict(X_test)
score = accuracy_score(Y_test, test_pred)
print(f"Tuned Test Set Accuracy : {accuracy_score(Y_test, test_pred) * 100} %\n\n")  
print()
print("accuracy_matrix")
print(confusion_matrix(Y_test,test_pred))
print()
report=classification_report(Y_test, test_pred)
print(f"Classification Report : \n\n{classification_report(Y_test, test_pred)}")
# dict_models["passive_aggresive"]=model_pa
# dict_accuracy[score]="passive_aggresive"

# dict_models_tfidf["passive_aggresive"]=model_pa
# dict_accuracy_report_tfidf["passive_aggresive"]=report

Tuned Test Set Accuracy : 67.48283752860412 %



accuracy_matrix
[[ 120 1347]
 [  74 2829]]

Classification Report : 

              precision    recall  f1-score   support

           0       0.62      0.08      0.14      1467
           1       0.68      0.97      0.80      2903

    accuracy                           0.67      4370
   macro avg       0.65      0.53      0.47      4370
weighted avg       0.66      0.67      0.58      4370



In [35]:
print(Y_test)

[1 1 1 ... 1 1 1]


In [36]:
X_train.shape

(8870, 50)

In [37]:
preprocess("my name is udit narang1243 Udit")

['my', 'name', 'is', 'udit', 'narang1243', 'udit']

In [38]:
def find_max_len(sentences):
    n=0
    for sentence in sentences:
        list_of_words=preprocess(sentence)
        if n<len(list_of_words):
            n=len(list_of_words)
    return n
        

In [39]:
max_train_len=find_max_len(df_train["tweet"])

In [40]:
max_train_len

103

In [41]:
X_train=np.zeros((len(df_train), 50*(max_train_len+10)))

In [42]:
len_embedding=X_train.shape[1]

In [43]:
def get_sentence_embedding(sentence, embeddings_dict):
    # While creating sentence vector, only consider words that have a key in the glove dict
    sentence_embedding=np.zeros(len_embedding)
    relevant_embeddings = [embeddings_dict[word] for word in preprocess(sentence) if word in embeddings_dict.keys()]
    if len(relevant_embeddings) > 0:
        for i in range(len(relevant_embeddings)):
            for j in range(50):
                sentence_embedding[i*50+j]=relevant_embeddings[i][j]
    return sentence_embedding

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
X_train, X_test, y_train, y_test = train_test_split(df_train["tweet"], df_train["label"], test_size=0.245, random_state=42)

In [48]:
X_train.shape

(6696,)

In [49]:
df_new=pd.DataFrame({"tweet": X_train, "label": y_train})

In [50]:
df_new.shape

(6696, 2)

In [52]:
df_new.to_csv("small_train_set.csv")